## 동행복권 회차별 로또 번호 요청 API

https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo=(원하는 회차 번호)

Concept
1. 회차별 로또 번호를 가져와서, DB에 적재한다.
2. 실행시키는 순간, 적재되어있는 회차를 조회하여 신규 회차가 존재할 경우 추가 적재
3. 

Response Info<br>
추첨회차:drwNo<br>
추첨일:drwNoDate<br>
번호1:drwtNo1<br>
번호2:drwtNo2<br>
번호3:drwtNo3<br>
번호4:drwtNo4<br>
번호5:drwtNo5<br>
번호6:drwtNo6<br>
보너스번호:bnusNo<br>
1등총액:firstAccumamnt<br>
1등당첨자수:firstPrzwnerCo<br>
1등당첨액:firstWinamnt<br>
판매금액:totSellamnt<br>
결과값:returnValue<br>

In [11]:
import requests
import pymysql
import pandas as pd

In [12]:
class LottoCash:
    def __init__(self, host, port, db, user, passwd):
        self.host=host
        self.port=port
        self.db=db
        self.user=user
        self.passwd=passwd
        self.lotto_url = "https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo="
        self.drwNo = 0     ## 회차
        
    def db_conn(self):
        try:
            conn = pymysql.connect(host=self.host, port=self.port, db=self.db, user=self.user, passwd=self.passwd)
            return conn
        except Exception as e:
            print(e)
            
    def get_drwNo(self, conn):
        try:
            cursor = conn.cursor()
            cursor.execute("""SELECT drwNo FROM lotto LIMIT1 ORDER BY drwNo desc """)
            df_result = pd.DataFrame(cursor.fetchall())
#             print("result = "+str(df_result[0][0]))

            if df_result.empty:
                self.drwNo = 1
            else:
                self.drwNo = df_result[0][0]+1
            print("수집할 회차 : "+str(self.drwNo))
        except Exception as e:
            conn.close()
            print(e)
            
    def get_lottoInfo(self):
        try:
            round_url = self.lotto_url+str(self.drwNo)    # 동행복권 로또API에 원하는 회차를 입력
            print(str(self.drwNo)+"회차 = "+round_url)
            result = requests.get(round_url)
            return result.json()
        except Exception as e:
            print(e)

    def set_lottoInfo(self, conn, round_lottoInfo):
        try:
            cursor = conn.cursor()
            sql = """INSERT INTO lotto (drwNo
                                        , drwNoDate
                                        , drwtNo1, drwtNo2, drwtNo3, drwtNo4, drwtNo5, drwtNo6, bnusNo
                                        , firstAccumamnt
                                        , firstPrzwnerCo
                                        , firstWinamnt
                                        , totSellamnt
                                        , returnValue) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
            datas = (round_lottoInfo.get('drwNo')
                    , round_lottoInfo.get('drwNoDate')
                    , round_lottoInfo.get('drwtNo1')
                    , round_lottoInfo.get('drwtNo2')
                    , round_lottoInfo.get('drwtNo3')
                    , round_lottoInfo.get('drwtNo4')
                    , round_lottoInfo.get('drwtNo5')
                    , round_lottoInfo.get('drwtNo6')
                    , round_lottoInfo.get('bnusNo')
                    , round_lottoInfo.get('firstAccumamnt')
                    , round_lottoInfo.get('firstPrzwnerCo')
                    , round_lottoInfo.get('firstWinamnt')
                    , round_lottoInfo.get('totSellamnt')
                    , round_lottoInfo.get('returnValue'))
            print(datas)
            cursor.execute(sql, datas)
            conn.commit()
        except Exception as e:
            print(e)
        finally:
            cursor.close()

In [13]:
lotto = LottoCash('localhost', 3306, 'lotto', 'administrator', '12qwaszx!@')

In [21]:
conn = lotto.db_conn()
lotto.get_drwNo(conn)

수집할 회차 : 999


In [23]:
while True:
    round_lottoInfo = lotto.get_lottoInfo()
    if "fail"!=round_lottoInfo.get('returnValue'):
        lotto.set_lottoInfo(conn, round_lottoInfo)    
        lotto.drwNo = lotto.drwNo+1
    else:
        break
    

999회차 = https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo=999


In [19]:
conn.close()